In [47]:
import numpy as np
from sklearn.cluster import MeanShift, KMeans
from sklearn import preprocessing, model_selection
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
df = pd.read_excel('titanic.xls')

original_df = pd.DataFrame.copy(df)##
df.drop(['body','name'], 1, inplace=True)
df.fillna(0,inplace=True)

In [49]:
def handle_non_numerical_data(df):
    
    # handling non-numerical data: must convert.
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        #print(column,df[column].dtype)
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            
            column_contents = df[column].values.tolist()
            #finding just the uniques
            unique_elements = set(column_contents)
            # great, found them. 
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    # creating dict that contains new
                    # id per unique string
                    text_digit_vals[unique] = x
                    x+=1
            # now we map the new "id" vlaue
            # to replace the string. 
            df[column] = list(map(convert_to_int,df[column]))

    return df

In [50]:
df = handle_non_numerical_data(df)
df.drop(['ticket','home.dest'], 1, inplace=True)
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])
clf = MeanShift()
clf.fit(X)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [51]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

In [52]:
original_df['cluster_group']=np.nan
original_df = original_df.copy()
#copy problem!!#
for i in range(len(X)):
    original_df.loc[i,'cluster_group'] = labels[i]

In [53]:
n_clusters_ = len(np.unique(labels))
survival_rates = {}
for i in range(n_clusters_):
    temp_df = original_df[ (original_df['cluster_group']==float(i)) ]
    #print(temp_df.head())

    survival_cluster = temp_df[  (temp_df['survived'] == 1) ]

    survival_rate = len(survival_cluster) / len(temp_df)
    #print(i,survival_rate)
    survival_rates[i] = survival_rate
    
print(survival_rates)

{0: 0.372, 1: 0.6666666666666666, 2: 1.0, 3: 0.1}


In [54]:
print(original_df[ (original_df['cluster_group']==1) ])

     pclass  survived                                               name  \
1         1         1                     Allison, Master. Hudson Trevor   
2         1         0                       Allison, Miss. Helen Loraine   
3         1         0               Allison, Mr. Hudson Joshua Creighton   
4         1         0    Allison, Mrs. Hudson J C (Bessie Waldo Daniels)   
10        1         0                             Astor, Col. John Jacob   
11        1         1  Astor, Mrs. John Jacob (Madeleine Talmadge Force)   
16        1         0                           Baxter, Mr. Quigg Edmond   
17        1         1    Baxter, Mrs. James (Helene DeLaudeniere Chaput)   
23        1         1                              Bidois, Miss. Rosalie   
24        1         1                                  Bird, Miss. Ellen   
35        1         1                           Bowen, Miss. Grace Scott   
55        1         1                          Carter, Miss. Lucile Polk   
56        1 

In [55]:
print(original_df[ (original_df['cluster_group']==0) ].describe())

            pclass     survived         age        sibsp        parch  \
count  1250.000000  1250.000000  989.000000  1250.000000  1250.000000   
mean      2.340000     0.372000   29.407735     0.485600     0.304800   
std       0.814451     0.483532   14.187561     1.048939     0.656843   
min       1.000000     0.000000    0.166700     0.000000     0.000000   
25%       2.000000     0.000000   21.000000     0.000000     0.000000   
50%       3.000000     0.000000   28.000000     0.000000     0.000000   
75%       3.000000     1.000000   38.000000     1.000000     0.000000   
max       3.000000     1.000000   80.000000     8.000000     4.000000   

              fare        body  cluster_group  
count  1249.000000  114.000000         1250.0  
mean     25.477148  161.991228            0.0  
std      28.780926   98.488173            0.0  
min       0.000000    1.000000            0.0  
25%       7.895800   70.500000            0.0  
50%      13.500000  165.500000            0.0  
75%   

In [56]:
cluster_0 = (original_df[ (original_df['cluster_group']==0) ])
cluster_0_fc = (cluster_0[ (cluster_0['pclass']==1) ])
print(cluster_0_fc.describe())

       pclass    survived         age       sibsp       parch       fare  \
count   274.0  274.000000  235.000000  274.000000  274.000000  274.00000   
mean      1.0    0.605839   39.442553    0.375912    0.200730   61.90844   
std       0.0    0.489564   13.971044    0.521624    0.476402   39.25586   
min       1.0    0.000000    4.000000    0.000000    0.000000    0.00000   
25%       1.0    0.000000   29.000000    0.000000    0.000000   29.77500   
50%       1.0    1.000000   39.000000    0.000000    0.000000   53.10000   
75%       1.0    1.000000   49.000000    1.000000    0.000000   79.65000   
max       1.0    1.000000   80.000000    2.000000    2.000000  227.52500   

             body  cluster_group  
count   30.000000          274.0  
mean   172.566667            0.0  
std     84.511449            0.0  
min     16.000000            0.0  
25%    114.000000            0.0  
50%    173.500000            0.0  
75%    242.250000            0.0  
max    307.000000            0.0  
